# Variáveis Dummies

    * São variáveis binárias representadas por 0 ou 1, indicando ausência ou presença respectivamente de atributos em variáveis categoricas. As máquinas só entendem números e não letras, por isso a necessidade de se converter variáveis do tipo texto em números que possam ser interpretados pelas máquinas. 
    
    

<img src='variaveis.png'>

Numéricos: Como proprio nome diz, são variaveis representadas por números:
   * Discreto: Representa algo naquele momento, número de chegada, numéro da casa e etc.
   * Continuo: Valor de algo continuo, PI, massa da terra etc.

Categórico: Representádo por algo que não é numérico:
   * Nominal: sexo - masculino e feminino (não há hierarquia)
   * Ordinal: desenvolvedor - junior, pleno, senior (há hierarquia)

In [79]:
import pandas as pd
dados = {'raça':['Husky', 'fila', 'golden', 'pastor', 'fila'], 'idade':[3, 7, 5, 6, 2], 'peso':[12.5, 11, 9, 10, 7]}
df = pd.DataFrame(dados)
display(df)

,raça,idade,peso
0,Husky,3,12.5
1,fila,7,11.0
2,golden,5,9.0
3,pastor,6,10.0
4,fila,2,7.0


In [88]:
# Verificando dados categoricos (object)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   idade   5 non-null      int64  
 1   raça_0  5 non-null      float64
 2   raça_1  5 non-null      float64
 3   raça_2  5 non-null      float64
 4   raça_3  5 non-null      float64
dtypes: float64(4), int64(1)
memory usage: 328.0 bytes


# Método Label Encoder

In [81]:
# Importando Label Encoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Aplicando o encoder
encoder = LabelEncoder().fit(df['raça'])
df['raça'] = encoder.transform(df['raça'])

display(df)

,raça,idade,peso
0,0,3,12.5
1,1,7,11.0
2,2,5,9.0
3,3,6,10.0
4,1,2,7.0


Note que agora cada raça esta sendo representada por um inteiro. fila = 1

# Método One-Hot Encoder

In [82]:
ht = OneHotEncoder()

X = ht.fit_transform(df.raça.values.reshape(-1,1)).toarray()

display(x)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

In [83]:
df_hot= pd.DataFrame(X, columns=['raça_'+str(int(i)) for i in range(df.shape[1]+1)])

df = pd.concat([df, df_hot], axis=1)

df = df.drop(["raça"], axis=1)
print(df)

   idade  peso  raça_0  raça_1  raça_2  raça_3
0      3  12.5     1.0     0.0     0.0     0.0
1      7  11.0     0.0     1.0     0.0     0.0
2      5   9.0     0.0     0.0     1.0     0.0
3      6  10.0     0.0     0.0     0.0     1.0
4      2   7.0     0.0     1.0     0.0     0.0


# Pandas GET_DUMMIES
    * Função pronta do pandas que converte variáveis categóricas usando método one-hot agilizando o processo. (descomente para usa-la, lembrando de aplicar ao dataframe sem tratmento dummie ou dará erro.)

In [84]:
# No lugar do one-hot podemos usar get_dummies do Pandas

# col_dummies = ['raça']
# df = pd.get_dummies(df, columns = col_dummies)
# display(df)

# MULTICOLINEARIDADE

In [85]:
# Nível de pontuação VIF
import statsmodels.api as sm

def pontuação_vif(df):
    vif_df = pd.DataFrame(columns = ['var', 'vif'])
    x_var_names = df.columns
    
    # Para o número total de colunas na tabela
    for i in range(0, x_var_names.shape[0]):
        y = df[x_var_names[i]]
        x = df[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        
        # Formula do calculo do vif
        vif = round(1/(1-r_squared),2)
        
        vif_df.loc[i] = [x_var_names[i], vif]
        
    return vif_df.sort_values(by = 'vif', axis=0, ascending=False, inplace=False)
        

In [86]:
'''
Note valores INF isso mostra uma correlação perfeita entre duas variáveis independentes. 
No caso de correlação perfeita, obtemos R2 =1, o que leva a 1/(1-R2) infinito.
Para resolver esse problema, precisamos descartar uma das variáveis do 
conjunto de dados que está causando essa multicolinearidade perfeita.
'''
pontuação_vif(df)

C:\Users\devda\AppData\Local\Temp\ipykernel_39544\892647007.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  vif = round(1/(1-r_squared),2)
C:\Users\devda\AppData\Local\Temp\ipykernel_39544\892647007.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  vif = round(1/(1-r_squared),2)
C:\Users\devda\AppData\Local\Temp\ipykernel_39544\892647007.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  vif = round(1/(1-r_squared),2)
C:\Users\devda\AppData\Local\Temp\ipykernel_39544\892647007.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  vif = round(1/(1-r_squared),2)
C:\Users\devda\AppData\Local\Temp\ipykernel_39544\892647007.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  vif = round(1/(1-r_squared),2)
C:\Users\devda\AppData\Local\Temp\ipykernel_39544\892647007.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  vif = round(1/(1-r_squared),2)


,var,vif
0,idade,inf
1,peso,inf
2,raça_0,inf
3,raça_1,inf
4,raça_2,inf
5,raça_3,inf


In [87]:
# Testando excluir colunas par diminuir a multicolinearidade
# OBS: Teste excluir a coluna 0 depois a 1, vc vai notar a diferenca
df = df.drop(df.columns[[1]], axis=1)
pontuação_vif(df)

,var,vif
2,raça_1,4.24
4,raça_3,3.88
3,raça_2,3.00
1,raça_0,1.72
0,idade,1.38


NOTA: <br>
excluindo a coluna 0 temos vif maiores que 5<br>
excluindo a coluna 1 todos os vifs ficamabaixo de 5